# 패키지

In [307]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

# 데이터

In [308]:
path1 = 'C:/Users/samsung/Desktop/수업자료/train.csv'
path2 = 'C:/Users/samsung/Desktop/수업자료/test.csv'
path3 = 'C:/Users/samsung/Desktop/수업자료/sample_submission.csv'


train = pd.read_csv(path1)
test = pd.read_csv(path2)
submission = pd.read_csv(path3)

In [309]:
train.shape, test.shape, submission.shape

((2952, 15), (1022, 14), (150, 2))

In [310]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [311]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


# 결측치 확인

In [312]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [313]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

# 컬럼명 변경

In [314]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [315]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적',
    '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형',
    '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

# 지역명 숫자로 매핑(제거)

In [316]:
# 한 컬럼에 각기 다른 숫자형 데이터로 변경한 후,
# 이 데이터 값을 머신러닝 알고리즘에 그대로 넣어 데이터를 예측 하라고 지시한다면
# 컴퓨터가 이들 값의 관계를 예상과 다르게 형성할 가능성이 있다

In [317]:
# local_map = {}

# for i,loc in enumerate(train['지역'].unique()):
#     local_map[loc] = i

# local_map

In [318]:
# train['지역'] = train['지역'].map(local_map)
# test['지역'] = test['지역'].map(local_map)

# 원핫인코딩을 통한 지역명 변경

In [319]:
train = pd.get_dummies(data = train, columns = ['지역'], prefix = '지역')
test = pd.get_dummies(data = test, columns = ['지역'], prefix = '지역')

# 전용면적을 5의 배수로 변경

In [320]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

# 전용면적 상/하한 적용

In [321]:
# 상한100, 하한 15

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [322]:
train['전용면적'].unique()

array([ 35.,  50.,  55.,  30.,  45.,  40.,  25.,  70.,  15.,  20., 100.,
        60.,  75.,  80.,  65.])

# 단지별 데이터 1차원으로 취합

In [323]:
seoul = train.지역_서울특별시.copy()
seoul

0       0
1       0
2       0
3       0
4       0
       ..
2947    0
2948    0
2949    0
2950    0
2951    0
Name: 지역_서울특별시, Length: 2952, dtype: uint8

In [324]:
del train['지역_서울특별시']

In [325]:
train['지역_서울특별시'] = seoul

In [328]:
test['지역_서울특별시'] = 0

In [329]:
test

,단지코드,총세대수,임대건물구분,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,...,지역_대전광역시,지역_부산광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도,지역_서울특별시
0,C1072,754,아파트,국민임대,35.0,116,14.0,H,22830000,189840,...,0,0,0,0,0,0,0,0,0,0
1,C1072,754,아파트,국민임대,45.0,30,14.0,A,36048000,249930,...,0,0,0,0,0,0,0,0,0,0
2,C1072,754,아파트,국민임대,45.0,112,14.0,H,36048000,249930,...,0,0,0,0,0,0,0,0,0,0
3,C1072,754,아파트,국민임대,45.0,120,14.0,H,36048000,249930,...,0,0,0,0,0,0,0,0,0,0
4,C1072,754,아파트,국민임대,50.0,60,14.0,H,43497000,296780,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,행복주택,35.0,126,38.0,L,-,-,...,0,0,0,0,0,0,0,0,0,0
1018,C2189,382,아파트,국민임대,25.0,96,45.0,H,6872000,106400,...,0,0,0,0,0,1,0,0,0,0
1019,C2189,382,아파트,국민임대,25.0,20,45.0,H,6872000,106400,...,0,0,0,0,0,1,0,0,0,0
1020,C2189,382,아파트,국민임대,35.0,202,45.0,H,13410000,144600,...,0,0,0,0,0,1,0,0,0,0


In [352]:
location = train.iloc[:,14:].columns.tolist()
location

['지역_강원도',
 '지역_경기도',
 '지역_경상남도',
 '지역_경상북도',
 '지역_광주광역시',
 '지역_대구광역시',
 '지역_대전광역시',
 '지역_부산광역시',
 '지역_세종특별자치시',
 '지역_울산광역시',
 '지역_전라남도',
 '지역_전라북도',
 '지역_제주특별자치도',
 '지역_충청남도',
 '지역_충청북도',
 '지역_서울특별시']

In [353]:
columns = ['단지코드', '총세대수', '공가수'] + location + [ '단지내주차면수','지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [354]:
columns

['단지코드',
 '총세대수',
 '공가수',
 '지역_강원도',
 '지역_경기도',
 '지역_경상남도',
 '지역_경상북도',
 '지역_광주광역시',
 '지역_대구광역시',
 '지역_대전광역시',
 '지역_부산광역시',
 '지역_세종특별자치시',
 '지역_울산광역시',
 '지역_전라남도',
 '지역_전라북도',
 '지역_제주특별자치도',
 '지역_충청남도',
 '지역_충청북도',
 '지역_서울특별시',
 '단지내주차면수',
 '지하철',
 '버스']

In [355]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [356]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:15, 26.67it/s]
150it [00:06, 24.03it/s]


In [357]:
new_test

,단지코드,총세대수,공가수,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,...,면적_40.0,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0
0,C1072,754.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1128,1354.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C1456,619.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C1840,593.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C1332,1297.0,11.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,349.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,C1266,596.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,C2152,120.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0
148,C1267,675.0,38.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,74.0,0.0,50.0,240.0,0.0,0.0,0.0,0.0,0.0


# 결측치 처리

In [366]:
new_train.isna().sum().to_frame()

,0
단지코드,0
총세대수,0
공가수,0
지역_강원도,0
지역_경기도,0
지역_경상남도,0
지역_경상북도,0
지역_광주광역시,0
지역_대구광역시,0
지역_대전광역시,0


In [359]:
new_train.지하철.value_counts()

0.0    359
1.0     38
2.0      5
3.0      1
Name: 지하철, dtype: int64

In [360]:
new_train.버스.mode()

0    2.0
dtype: float64

In [361]:
import numpy as np

new_train.지하철 = new_train.지하철.replace(np.nan, 0)

In [362]:
new_train.버스 = new_train.버스.replace(np.nan, 2)

In [367]:
new_test.isna().sum().to_frame()

,0
단지코드,0
총세대수,0
공가수,0
지역_강원도,0
지역_경기도,0
지역_경상남도,0
지역_경상북도,0
지역_광주광역시,0
지역_대구광역시,0
지역_대전광역시,0


In [364]:
new_test.지하철.value_counts()

0.0    134
1.0      8
2.0      3
Name: 지하철, dtype: int64

In [365]:
new_test.지하철 = new_test.지하철.replace(np.nan, 0)

# 학습

In [368]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [369]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [370]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

# 추론 및 제출

In [371]:
pred = model.predict(x_test)

In [372]:
submission['num'] = pred

In [373]:
submission

,code,num
0,C1072,683.57
1,C1128,1201.78
2,C1456,490.76
3,C1840,533.51
4,C1332,1063.37
...,...,...
145,C2456,273.17
146,C1266,493.84
147,C2152,29.16
148,C1267,458.40


In [374]:
submission.to_csv('preparing.csv',index=False)